# Get a LSST Simulated Cadence for Single Field
___


___

In [ ]:
"""

"""
%matplotlib inline
# %matplotlib notebook

from __future__ import print_function

try:
    from importlib import reload
except:
    pass


from matplotlib import pyplot as plt
import matplotlib.colors as mpl_colors 

import os
import sys
import warnings

import numpy as np
import pandas as pd
import astropy as ap
import sfdmap

import sqlite3
from sqlalchemy import create_engine

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table

import lsst_tools.utils as utils 
import lsst_tools.utils.colours as colours 

import pyCoCo as pccsims
import pycoco as pcc

Connect to .db file that contains the opsim output, read into a dataframe, `opsimdf`

In [ ]:
# %%timeit ## 1 loop, best of 3: 1min 31s per loop 
opsimdbpath = os.environ.get('OPSIMDBPATH')
print(opsimdbpath)
# opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/astro_lsst_01_1068_sqlite.db"
opsimdbpath = "/Users/berto/data/LSST/OpSimOutputDBs/minion_1016_sqlite.db"

conn = create_engine('sqlite:///'+opsimdbpath, echo = False)
# opsimdf = pd.read_sql_table('Summary', con=conn)
opsimdf = pd.read_sql_query(sql = "SELECT * FROM Summary WHERE night > 365 AND night < 731", con = conn) ## JUST YEAR 2

Check that the db looks as we expect

In [ ]:
opsimdf.head()

Connecting to `.db` takes ages (~1min), and is a pain if you mess up, so create a 'working' instance to fiddle with

In [ ]:
working_df = opsimdf

In [ ]:
working_df = utils.find_MW_extinction(working_df)

allfields_df = utils.get_field_corners(
    pd.DataFrame(working_df.drop_duplicates("fieldID"))[['fieldID', 'fieldRA', 'fieldDec', 'EBV_MW']])

## Choosing Position and Identifying Fields
___

Choose a fieldID to to give us a position.

### Deep Drilling Fields:
___

* minion2016: 1427,744,2412,290,2786
* astro_lsst_01_1068: 1427,744,2412,290,2786
* Fake_Rolling: 290,2786

In [ ]:
chosen_fieldID = 2786

field_df = allfields_df.loc[allfields_df['fieldID'] == chosen_fieldID]

In [ ]:
field_df = utils.field_df_to_pos_df(field_df)
field_df.head()

## Extinction

Get the Milky Way extinction along the line of site towards the SNe. The working_df contains the EBV_MW at the field centre.

In [ ]:
extinction = field_df["EBV_MW"].values[0]
print(extinction)

## Simulating
------
inputs:

* **`snname`**
* **`redshift`**
* **`absmag offset`**
* **`EBV MW`**
* **`EBV Host`**
* **`Rv`**
* **`MJD at Peak`**

* **`MJD to simulate`**
* **`filters to simulate`**
----

#### Epochs and filters

* pull MJD and filters from df where fieldID matches 

In [ ]:
mjd_to_sim = working_df["expMJD"][working_df["fieldID"].isin(field_df["fieldID"].values)].values
limiting_mags = working_df["fiveSigmaDepth"][working_df["fieldID"].isin(field_df["fieldID"].values)].values

filters_to_sim = working_df["filter"][working_df["fieldID"].isin(field_df["fieldID"].values)].values
filters_to_sim = np.array([utils.b('LSST_'+x) for x in filters_to_sim])

In [ ]:
filter_path =  pcc._default_filter_dir_path
coco_root_path = pcc._default_coco_dir_path 

coco = pccsims.pyCoCo(utils.b(filter_path), utils.b(coco_root_path))

In [ ]:
snname = "SN2007uy"

sn = pcc.SNClass(snname)

In [ ]:
z_obs = sn.info["z_obs"]

host_EBV = 0.0

extinction = 0.0
MW_EBV = extinction

print(extinction)

# mjdmax = 60307.314753999999
mjdmax = 59580 + 1.* 365. ## DDF 2786
# mjdmax = 59580 + 1.5* 365. ## WFD 550
# mjdmax = 59580 + 1.3* 365. ## WFD 2297

# field_df = pd.DataFrame({"fieldID" : [2786]})

In [ ]:
verbose = False
if verbose:
    print(mjdmax)
    print(mjd_to_sim)
    print(filters_to_sim)

In [ ]:
flux, flux_err = coco.simulate(utils.b(snname), 
                    z_obs, 0.0, MW_EBV, host_EBV, 3.1, 
                    mjdmax, mjd_to_sim, 
                    filters_to_sim)

In [ ]:
flux, flux_err

In [ ]:
print(np.nanmax(flux))
print(mjdmax)

We can now look at the specphot directly

In [ ]:
phase, sp_flux = coco.spec_photometry(b"SN2007uy", z = z_obs, flt = b"LSST_r")

p = pcc.PhotometryClass()
p.load_table(pcc.utils.simulate_out_to_ap_table(phase + mjdmax, sp_flux, np.ones_like(sp_flux)*1e-17,
                                                np.array([b"LSST_r" for i in range(len(sp_flux))])))

p.plot()

In [ ]:
# p.save(filename = "SN2007uy_sim_LSST_r.dat", path = "/Users/berto/projects/LSST/cadence/")

In [ ]:
snname = "SN2007uy_LSST_r_specphot"

sn = pcc.SNClass(snname)
phot_path = "/Users/berto/projects/LSST/cadence/SN2007uy_sim_LSST_r.dat"

sn.load_phot(path = phot_path) 
recon_filename = os.path.abspath(os.path.join(str(coco_root_path), "recon/SN2007uy_sim_LSST_r.dat"))
sn.get_lcfit(recon_filename)

sn.plot_lc(multiplot = False)

In [ ]:
flux_limit = 1e-20

p = pcc.PhotometryClass()
phot_table = pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim)
phot_table = phot_table[np.where(phot_table["flux"] > flux_limit)]
p.load_table(phot_table)

In [ ]:
# p.plot(["LSST_r"], xlim = [59995, 60100])
p.plot(xlim = [59900, 60100])
# p.plot(["LSST_r", "LSST_g"], xlim = [59900, 60100])

## With Extinction - Priors need to respond?

In [ ]:
snname = "SN2007uy"

sn = pcc.SNClass(snname)

In [ ]:
z_obs = sn.info["z_obs"]

host_EBV = 0.0

MW_EBV = extinction
print(extinction)
# MW_EBV = 0.0

# mjdmax = 60307.314753999999
mjdmax = 59580 + 1.* 365. ## DDF 2786
# mjdmax = 59580 + 1.5* 365. ## WFD 550
# mjdmax = 59580 + 1.3* 365. ## WFD 2297

# field_df = pd.DataFrame({"fieldID" : [2786]})

In [ ]:
verbose = False
if verbose:
    print(mjdmax)
    print(mjd_to_sim)
    print(filters_to_sim)

In [ ]:
flux, flux_err = coco.simulate(utils.b(snname), 
                    z_obs, 0.0, MW_EBV, host_EBV, 3.1, 
                    mjdmax, mjd_to_sim, 
                    filters_to_sim)

In [ ]:
flux, flux_err

In [ ]:
print(np.nanmax(flux))
print(mjdmax)

We can now look at the specphot directly

In [ ]:
phase, sp_flux = coco.spec_photometry(b"SN2007uy", z = z_obs, flt = b"LSST_r")


In [ ]:
plt.scatter(phase, sp_flux)
plt.ylim(0, np.nanmax(flux)*1.05)

In [ ]:
# reload(pcc.classes)

In [ ]:
flux_limit = 1e-20

p = pcc.PhotometryClass()
phot_table = pcc.utils.simulate_out_to_ap_table(mjd_to_sim, flux, flux_err, filters_to_sim)
phot_table = phot_table[np.where(phot_table["flux"] > flux_limit)]
p.load_table(phot_table)

p.save(filename = "SN2007uy_sim_LSST_r_EBV.dat", path = "/Users/berto/projects/LSST/cadence/")

In [ ]:
# p.plot(["LSST_r"], xlim = [59995, 60100])
p.plot(xlim = [59900, 60100])
# p.plot(["LSST_r", "LSST_g"], xlim = [59900, 60100])